In [ ]:
# Paso 1: Librerías necesarias
import numpy as np
import pickle, tarfile, os
from urllib.request import urlretrieve

# Paso 2: Cargar CIFAR-10
def load_CIFAR10(path):
    url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    if not os.path.exists(path): os.makedirs(path)
    tgz_path = os.path.join(path, 'cifar-10-python.tar.gz')
    if not os.path.exists(tgz_path):
        print("Descargando CIFAR-10...")
        urlretrieve(url, tgz_path)
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path)
    def load_batch(file):
        with open(file, 'rb') as f:
            d = pickle.load(f, encoding='bytes')
            return d[b'data'], d[b'labels']
    Xs, Ys = [], []
    for i in range(1, 6):
        X, Y = load_batch(os.path.join(path, 'cifar-10-batches-py', f'data_batch_{i}'))
        Xs.append(X); Ys.append(Y)
    Xtr = np.concatenate(Xs); Ytr = np.concatenate(Ys)
    Xte, Yte = load_batch(os.path.join(path, 'cifar-10-batches-py', 'test_batch'))
    return Xtr, Ytr, Xte, np.array(Yte)

# Paso 3: Implementar Nearest Neighbor
class NearestNeighbor:
    def train(self, X, y): self.Xtr, self.ytr = X, y
    def predict(self, X):
        Ypred = np.zeros(X.shape[0], dtype=self.ytr.dtype)
        for i in range(X.shape[0]):
            dists = np.sum(np.abs(self.Xtr - X[i, :]), axis=1)
            Ypred[i] = self.ytr[np.argmin(dists)]
            if i % 100 == 0: print(f'Procesando {i}/{X.shape[0]}')
        return Ypred

# Paso 4: Ejecutar todo
Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/')
Xtr_rows = Xtr.reshape(Xtr.shape[0], -1)[:1000]  # usar solo 1000 imágenes para rapidez
Ytr = Ytr[:1000]
Xte_rows = Xte.reshape(Xte.shape[0], -1)[:100]   # usar solo 100 imágenes de prueba
Yte = Yte[:100]

nn = NearestNeighbor()
nn.train(Xtr_rows, Ytr)
Yte_predict = nn.predict(Xte_rows)

print('accuracy: %f' % (np.mean(Yte_predict == Yte)))



Procesando 0/100
accuracy: 0.240000


In [ ]:
# Paso 1: Librerías necesarias
import numpy as np
import pickle, tarfile, os
from urllib.request import urlretrieve
from collections import Counter

# Paso 2: Cargar CIFAR-10
def load_CIFAR10(path):
    url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    if not os.path.exists(path): os.makedirs(path)
    tgz_path = os.path.join(path, 'cifar-10-python.tar.gz')
    if not os.path.exists(tgz_path):
        print("Descargando CIFAR-10...")
        urlretrieve(url, tgz_path)
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path)
    def load_batch(file):
        with open(file, 'rb') as f:
            d = pickle.load(f, encoding='bytes')
            return d[b'data'], d[b'labels']
    Xs, Ys = [], []
    for i in range(1, 6):
        X, Y = load_batch(os.path.join(path, 'cifar-10-batches-py', f'data_batch_{i}'))
        Xs.append(X); Ys.append(Y)
    Xtr = np.concatenate(Xs); Ytr = np.concatenate(Ys)
    Xte, Yte = load_batch(os.path.join(path, 'cifar-10-batches-py', 'test_batch'))
    return Xtr, Ytr, Xte, np.array(Yte)

# Paso 3: Implementar Nearest Neighbor mejorado (L2 + k-NN)
class NearestNeighbor:
    def train(self, X, y):
        self.Xtr = X
        self.ytr = y

    def predict(self, X, k=3):
        Ypred = np.zeros(X.shape[0], dtype=self.ytr.dtype)
        for i in range(X.shape[0]):
            dists = np.sqrt(np.sum((self.Xtr - X[i, :]) ** 2, axis=1))  # L2
            k_indices = np.argsort(dists)[:k]
            k_labels = self.ytr[k_indices]
            Ypred[i] = Counter(k_labels).most_common(1)[0][0]
            if i % 50 == 0:
                print(f'Procesando {i}/{X.shape[0]}')
        return Ypred

# Paso 4: Ejecutar todo
Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/')
Xtr_rows = Xtr.reshape(Xtr.shape[0], -1)[:5000]
Ytr = Ytr[:5000]
Xte_rows = Xte.reshape(Xte.shape[0], -1)[:500]
Yte = Yte[:500]

# Paso 5: Normalizar los datos (media 0, varianza 1)
mean = np.mean(Xtr_rows, axis=0)
std = np.std(Xtr_rows, axis=0)
Xtr_rows = (Xtr_rows - mean) / (std + 1e-7)
Xte_rows = (Xte_rows - mean) / (std + 1e-7)

# Paso 6: Entrenar y predecir
nn = NearestNeighbor()
nn.train(Xtr_rows, Ytr)
Yte_predict = nn.predict(Xte_rows, k=3)

# Paso 7: Calcular precisión
accuracy = np.mean(Yte_predict == Yte)
print(f'Precisión con k-NN (k=3) y L2: {accuracy:.4f}')



Procesando 0/500
Procesando 50/500
Procesando 100/500
Procesando 150/500
Procesando 200/500
Procesando 250/500
Procesando 300/500
Procesando 350/500
Procesando 400/500
Procesando 450/500
Precisión con k-NN (k=3) y L2: 0.2620


In [ ]:
# Paso 1: Librerías necesarias
import numpy as np
import pickle, tarfile, os
from urllib.request import urlretrieve
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

# Paso 2: Cargar CIFAR-10
def load_CIFAR10(path):
    url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    if not os.path.exists(path): os.makedirs(path)
    tgz_path = os.path.join(path, 'cifar-10-python.tar.gz')
    if not os.path.exists(tgz_path):
        print("Descargando CIFAR-10...")
        urlretrieve(url, tgz_path)
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path)
    def load_batch(file):
        with open(file, 'rb') as f:
            d = pickle.load(f, encoding='bytes')
            return d[b'data'], d[b'labels']
    Xs, Ys = [], []
    for i in range(1, 6):
        X, Y = load_batch(os.path.join(path, 'cifar-10-batches-py', f'data_batch_{i}'))
        Xs.append(X); Ys.append(Y)
    Xtr = np.concatenate(Xs); Ytr = np.concatenate(Ys)
    Xte, Yte = load_batch(os.path.join(path, 'cifar-10-batches-py', 'test_batch'))
    return Xtr, Ytr, Xte, np.array(Yte)

# Paso 3: Cargar datos y preprocesar
print("Cargando CIFAR-10...")
Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/')

# Reducimos el tamaño para acelerar ejecución (puedes aumentar si deseas)
Xtr = Xtr[:10000]
Ytr = Ytr[:10000]
Xte = Xte[:1000]
Yte = Yte[:1000]

# Aplanar imágenes
Xtr_rows = Xtr.reshape(Xtr.shape[0], -1)
Xte_rows = Xte.reshape(Xte.shape[0], -1)

# Normalización
mean = np.mean(Xtr_rows, axis=0)
std = np.std(Xtr_rows, axis=0)
Xtr_rows = (Xtr_rows - mean) / (std + 1e-7)
Xte_rows = (Xte_rows - mean) / (std + 1e-7)

# Reducción opcional de dimensión con PCA (puedes desactivarla si quieres)
print("Aplicando PCA...")
pca = PCA(n_components=200)
Xtr_pca = pca.fit_transform(Xtr_rows)
Xte_pca = pca.transform(Xte_rows)

# Paso 4: Entrenar SVM
print("Entrenando SVM...")
clf = SVC(kernel='rbf', C=10, gamma='scale')  # Puedes ajustar C y gamma
clf.fit(Xtr_pca, Ytr)

# Paso 5: Predecir y evaluar
print("Prediciendo...")
Ypred = clf.predict(Xte_pca)
accuracy = accuracy_score(Yte, Ypred)

print(f'Precisión final con SVM + PCA: {accuracy:.4f}')

Cargando CIFAR-10...
Aplicando PCA...
Entrenando SVM...
Prediciendo...
Precisión final con SVM + PCA: 0.4740
